In [ ]:
!pip install prince

In [3]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

# 차원축소
from sklearn.preprocessing import *
from sklearn.decomposition import PCA
import prince


In [4]:
data = pd.read_csv('./data_to_model/세대원_상가_제로_유치원_환경_도시농업_초등학교_사업체_채식_인구수_연령별_카페_유동인구_매출_sum.csv', index_col=0)
data.fillna(0, inplace=True)
data.head(3)

,2023년09월_전체세대,2023년09월_1인세대,2023년09월_2인세대,2023년09월_3인세대,2023년09월_4인세대,2023년09월_5인세대,2023년9월_6인가구이상,채소가게수,제로웨이스트 가게수,유치원수,...,2_40대,2_50대,2_60대,2_70대,2_80대,2_90대,2_기타,카페수,승하차인원_버스_지하철,1점포당_1년_매출액
가락1동,9836,1633,2492,2706,2438,449,118,10.0,0.0,1.0,...,4892,3948,3832,1672,478,94,4,43.0,184724.0,1.175032e+08
가락2동,12375,3568,2746,2825,2641,453,142,3.0,0.0,4.0,...,5304,5224,4686,1834,840,182,8,74.0,190052.0,3.372717e+07
가락본동,11667,4854,2640,2123,1567,385,98,9.0,2.0,2.0,...,3520,4500,4002,1718,768,132,4,99.0,273627.0,3.056880e+07


In [5]:
df = data.copy()
df.columns

Index(['2023년09월_전체세대', '2023년09월_1인세대', '2023년09월_2인세대', '2023년09월_3인세대',
       '2023년09월_4인세대', '2023년09월_5인세대', '2023년9월_6인가구이상', '채소가게수',
       '제로웨이스트 가게수', '유치원수', '거주인구대비취약자수', '거주인구대비취약자백분위수', '미세먼지지수',
       '미세먼지백분위', '행정구역면적', '공원수', '공원율', '도시농업(프로그램수)', '도시농업(시설수)', '초등학교수',
       '사업체수 (개)', '종사자수 (명)', '채식식당수', '인구수', '1_0대', '1_10대', '1_20대',
       '1_30대', '1_40대', '1_50대', '1_60대', '1_70대', '1_80대', '1_90대', '1_기타',
       '2_0대', '2_10대', '2_20대', '2_30대', '2_40대', '2_50대', '2_60대', '2_70대',
       '2_80대', '2_90대', '2_기타', '카페수', '승하차인원_버스_지하철', '1점포당_1년_매출액'],
      dtype='object')

In [6]:
# 인구 범위 축소하기
df['남자_10대이하'] = df['1_0대'] + df['1_10대']
df['남자_20_30대'] = df['1_20대'] + df['1_30대']
df['남자_40_60대'] = df['1_40대'] + df['1_50대'] + df['1_60대']
df['남자_70대이상'] = df['1_70대'] + df['1_80대'] + df['1_90대'] + df['1_기타']

df['여자_10대이하'] = df['2_0대'] + df['2_10대']
df['여자_20_30대'] = df['2_20대'] + df['2_30대']
df['여자_40_60대'] = df['2_40대'] + df['2_50대'] + df['2_60대']
df['여자_70대이상'] = df['2_70대'] + df['2_80대'] + df['2_90대'] + df['2_기타']

# 유치원, 초등학교 수 합치기
df['유치원_초등학교'] = df['유치원수'] + df['초등학교수']

In [86]:
df.columns

Index(['2023년09월_전체세대', '2023년09월_1인세대', '2023년09월_2인세대', '2023년09월_3인세대',
       '2023년09월_4인세대', '2023년09월_5인세대', '2023년9월_6인가구이상', '채소가게수',
       '제로웨이스트 가게수', '유치원수', '거주인구대비취약자수', '거주인구대비취약자백분위수', '미세먼지지수',
       '미세먼지백분위', '행정구역면적', '공원수', '공원율', '도시농업(프로그램수)', '도시농업(시설수)', '초등학교수',
       '사업체수 (개)', '종사자수 (명)', '채식식당수', '인구수', '1_0대', '1_10대', '1_20대',
       '1_30대', '1_40대', '1_50대', '1_60대', '1_70대', '1_80대', '1_90대', '1_기타',
       '2_0대', '2_10대', '2_20대', '2_30대', '2_40대', '2_50대', '2_60대', '2_70대',
       '2_80대', '2_90대', '2_기타', '카페수', '승하차인원_버스_지하철', '1점포당_1년_매출액',
       '남자_10대이하', '남자_20_30대', '남자_40_60대', '남자_70대이상', '여자_10대이하',
       '여자_20_30대', '여자_40_60대', '여자_70대이상', '유치원_초등학교'],
      dtype='object')

In [27]:
population_feature = ['2023년09월_1인세대', '남자_10대이하', '남자_20_30대', '남자_40_60대', '남자_70대이상', '여자_10대이하',
       '여자_20_30대', '여자_40_60대', '여자_70대이상']

# infra_feature = ['공원수', '도시농업(프로그램수)', '도시농업(시설수)']
# demand_feature = ['채소가게수', '카페수', '채식식당수', '초등학교수', '유치원수']
environ_featrue = ['미세먼지지수', '미세먼지백분위', '거주인구대비취약자수', '공원수', '도시농업(프로그램수)', '도시농업(시설수)']
market_feature = ['1점포당_1년_매출액', '승하차인원_버스_지하철', '종사자수 (명)', '사업체수 (개)', '채소가게수', '카페수', '채식식당수', '초등학교수', '유치원수']

In [28]:
len(population_feature) + len(environ_featrue) + len(market_feature)

24

In [7]:
# # 인구 합치기 전, 스케일링
# scaled_data2 = StandardScaler().fit_transform(np.log1p(data))
# data_scale_log2=pd.DataFrame(data=scaled_data2, columns=data.columns)

In [24]:
# 인구 합친 후, 스케일링 (로그, 스탠다드)
scale_kind = 'log_standard'
scaled_data = StandardScaler().fit_transform(np.log1p(df))
data_scale_log=pd.DataFrame(data=scaled_data, columns=df.columns)
data_scale_log.head(5)

,2023년09월_전체세대,2023년09월_1인세대,2023년09월_2인세대,2023년09월_3인세대,2023년09월_4인세대,2023년09월_5인세대,2023년9월_6인가구이상,채소가게수,제로웨이스트 가게수,유치원수,...,1점포당_1년_매출액,남자_10대이하,남자_20_30대,남자_40_60대,남자_70대이상,여자_10대이하,여자_20_30대,여자_40_60대,여자_70대이상,유치원_초등학교
0,0.082020,-1.232252,0.298329,0.868037,1.054325,0.838395,0.760275,1.691391,-0.602463,-0.648543,...,1.762880,1.244875,0.477965,0.643055,0.257813,1.232333,0.592844,0.683955,-0.122153,-0.018742
1,0.481592,-0.104676,0.458849,0.932058,1.161431,0.850548,1.040792,0.284733,-0.602463,1.121062,...,0.077817,1.196505,0.743152,0.933382,0.512298,1.129477,0.760896,0.960985,0.257900,1.034959
2,0.379076,0.339458,0.393743,0.507109,0.462467,0.627721,0.479326,1.558860,1.561220,0.134520,...,-0.054924,0.417238,0.693772,0.641965,0.340402,0.439900,0.612464,0.604165,0.119354,0.401415
3,-0.750427,0.215361,-1.232976,-1.958958,-2.224547,-2.630133,-3.565654,0.848543,-0.602463,-1.987197,...,-1.760553,-2.918667,-1.585274,-2.404177,-1.323867,-3.141218,-2.004816,-2.817779,-1.522056,-1.323869
4,1.139489,1.876298,0.132325,-0.618365,-0.893192,-0.920683,-0.943036,1.248573,-0.602463,-0.648543,...,1.089504,-0.733127,1.503474,0.313347,-0.142810,-0.709616,1.289160,-0.243285,-0.422832,-1.323869


In [ ]:
# # 인구 합친 후, 스케일링 (min-max)
# scaled_data = MinMaxScaler().fit_transform(df)
# data_scale_log=pd.DataFrame(data=scaled_data, columns=df.columns)
# data_scale_log

In [ ]:
# # 인구 합친 후, 스케일링 (log, min-max)
# scaled_data = MinMaxScaler().fit_transform(np.log1p(df))
# data_scale_log=pd.DataFrame(data=scaled_data, columns=df.columns)
# data_scale_log

#### 인구특성



In [9]:
population_feature = ['2023년09월_1인세대', '남자_10대이하', '남자_20_30대', '남자_40_60대', '남자_70대이상', '여자_10대이하',
       '여자_20_30대', '여자_40_60대', '여자_70대이상']

pca = PCA(n_components=0.95)
population_scale_data = data_scale_log[population_feature]
pca.fit(population_scale_data)
pca_population = pd.DataFrame((pca.transform(population_scale_data)))

num_of_principal = pca_population.shape[1] 
print('주성분의 개수:', num_of_principal)

주성분의 개수: 3


In [10]:
print('주성분의 설명력')
for i in range(num_of_principal) :
    print(f"제 {i+1}주성분 :", pca.explained_variance_ratio_[i])
    
pd.DataFrame(data=pca.components_,columns=population_scale_data.columns).T

주성분의 설명력
제 1주성분 : 0.8243068415827797
제 2주성분 : 0.11777893165341181
제 3주성분 : 0.037538521129754754


,0,1,2
2023년09월_1인세대,-0.248212,0.691402,0.109357
남자_10대이하,-0.323765,-0.422133,0.231143
남자_20_30대,-0.344253,0.268372,0.266949
남자_40_60대,-0.358644,-0.102116,-0.036978
남자_70대이상,-0.345005,0.038929,-0.550983
여자_10대이하,-0.323410,-0.419514,0.258925
여자_20_30대,-0.342574,0.232359,0.395523
여자_40_60대,-0.357236,-0.168928,-0.031918
여자_70대이상,-0.343270,0.035784,-0.577844


In [11]:
population_columns = ['인구가 비교적 적은 지역', '1인 세대 및 20~30대 남녀가 많은 지역', '미취학 및 초등학생, 젊은 세대가 많은 지역']

---
인구 합치기 전

In [ ]:
population_feature2 = ['2023년09월_전체세대', '2023년09월_1인세대', '2023년09월_2인세대', '2023년09월_3인세대',
       '2023년09월_4인세대', '2023년09월_5인세대', '2023년9월_6인가구이상', '인구수', '1_0대', '1_10대', '1_20대', '1_30대', '1_40대', '1_50대', '1_60대',
       '1_70대', '1_80대', '1_90대', '1_기타', '2_0대', '2_10대', '2_20대', '2_30대',
       '2_40대', '2_50대', '2_60대', '2_70대', '2_80대', '2_90대', '2_기타']

pca = PCA(n_components=0.9)
population_scale_data2 = data_scale_log2[population_feature2]
pca.fit(population_scale_data2)
pca_population2 = pd.DataFrame((pca.transform(population_scale_data2)))

num_of_principal = pca_population2.shape[1] 
print('주성분의 개수:', num_of_principal)

In [ ]:
print('주성분의 설명력')
for i in range(num_of_principal) :
    print(f"제 {i+1}주성분 :", pca.explained_variance_ratio_[i])
    
pd.DataFrame(data=pca.components_,columns=population_scale_data2.columns).T

---

### 환경 관련 인프라 특성

In [12]:
environ_feature = ['미세먼지지수', '미세먼지백분위', '거주인구대비취약자수', '공원수', '도시농업(프로그램수)', '도시농업(시설수)']

pca = PCA(n_components=0.85)
environ_scale_data = data_scale_log[environ_feature]
pca.fit(environ_scale_data)
pca_environ = pd.DataFrame((pca.transform(environ_scale_data)))

num_of_principal = pca_environ.shape[1] 
print('주성분의 개수:', num_of_principal)

주성분의 개수: 3


In [13]:
print('주성분의 설명력')
for i in range(num_of_principal) :
    print(f"제 {i+1}주성분 :", pca.explained_variance_ratio_[i])
    
pd.DataFrame(data=pca.components_,columns=environ_scale_data.columns).T

주성분의 설명력
제 1주성분 : 0.34494991794817903
제 2주성분 : 0.311309010781767
제 3주성분 : 0.21163370812189125


,0,1,2
미세먼지지수,0.272322,0.593267,-0.225943
미세먼지백분위,0.240552,0.561226,-0.336481
거주인구대비취약자수,0.035272,-0.455978,-0.528951
공원수,0.203339,-0.245971,-0.669930
도시농업(프로그램수),0.645616,-0.168277,0.230092
도시농업(시설수),0.639191,-0.190590,0.232793


In [14]:
environ_column = ['도시농업 체험 프로그램이 많은 지역', '공기질 오염이 심한 지역', '공원녹지가 부족한 지역']

### 상권 특성

In [15]:
market_feature = ['1점포당_1년_매출액', '승하차인원_버스_지하철', '종사자수 (명)', '사업체수 (개)', '채소가게수', '카페수', '채식식당수', '초등학교수', '유치원수']

pca = PCA(n_components=0.85)
market_scale_data = data_scale_log[market_feature]
pca.fit(market_scale_data)
pca_market = pd.DataFrame((pca.transform(market_scale_data)))

num_of_principal = pca_market.shape[1] 
print('주성분의 개수:', num_of_principal)

주성분의 개수: 5


In [16]:
print('주성분의 설명력')
for i in range(num_of_principal) :
    print(f"제 {i+1}주성분 :", pca.explained_variance_ratio_[i])
    
pd.DataFrame(data=pca.components_,columns=market_scale_data.columns).T

주성분의 설명력
제 1주성분 : 0.4424658409818186
제 2주성분 : 0.17119352080163067
제 3주성분 : 0.1176347578222602
제 4주성분 : 0.09620794430131395
제 5주성분 : 0.05246297849188734


,0,1,2,3,4
1점포당_1년_매출액,-0.122757,0.000869,-0.690619,0.706112,-0.064832
승하차인원_버스_지하철,-0.398226,0.090381,-0.009250,-0.007641,0.082025
종사자수 (명),-0.464186,-0.024304,-0.052965,-0.088007,0.101034
사업체수 (개),-0.457300,-0.019667,0.128240,0.001495,0.043630
채소가게수,-0.173291,-0.048152,0.673122,0.642310,0.070056
카페수,-0.465254,-0.038941,0.024518,-0.116538,-0.115328
채식식당수,-0.389649,-0.111004,-0.184323,-0.254739,-0.177768
초등학교수,-0.053689,0.696772,-0.077889,-0.038565,0.664129
유치원수,-0.023565,0.699432,0.099865,0.032815,-0.697195


In [17]:
market_column = ['사업체 및 유동인구가 적은 지역', '초등학교 및 유치원이 많은 지역', '채소 가게 소비가 많은 지역', '상가 매출이 많은 곳','초등학교가 많은 지역']

## 차원축소 결과

In [18]:
pca_result =pd.concat([pca_population, pca_environ, pca_market],axis=1)

In [19]:
pca_result.columns= population_columns + environ_column + market_column
pca_result.index = data.index
pca_result.head()

,인구가 비교적 적은 지역,1인 세대 및 20~30대 남녀가 많은 지역,"미취학 및 초등학생, 젊은 세대가 많은 지역",도시농업 체험 프로그램이 많은 지역,공기질 오염이 심한 지역,공원녹지가 부족한 지역,사업체 및 유동인구가 적은 지역,초등학교 및 유치원이 많은 지역,채소 가게 소비가 많은 지역,상가 매출이 많은 곳,초등학교가 많은 지역
가락1동,-1.385345,-1.803981,0.717072,-0.502649,0.088431,-0.199792,-0.491026,0.026846,0.131241,2.564634,1.186516
가락2동,-2.186509,-0.903525,0.560421,-0.129250,-0.448272,-1.514910,-0.678847,1.198229,0.318815,0.186852,-0.318566
가락본동,-1.414738,0.052435,0.375364,-0.146461,-0.225765,-1.256795,-2.658041,0.262004,0.903382,0.324795,0.300484
가리봉동,5.988013,2.422976,-0.892778,-0.980968,0.832861,1.690127,2.110154,-1.382596,1.742615,-0.177782,1.688921
가산동,-0.789125,2.595896,1.082431,-0.380187,0.636643,-0.011804,-6.147138,-2.132169,0.081840,0.490467,-0.787168


In [22]:
pca_result['사업체 및 유동인구가 적은 지역'].nsmallest(5)

역삼1동          -6.999186
종로1.2.3.4가동   -6.860880
여의동           -6.681952
가산동           -6.147138
서교동           -5.839454
Name: 사업체 및 유동인구가 적은 지역, dtype: float64

In [23]:
pca_result['사업체 및 유동인구가 적은 지역'].nlargest(5)

둔촌1동    8.280829
개포1동    4.950309
창신3동    4.920214
반포본동    4.737938
이촌2동    4.554871
Name: 사업체 및 유동인구가 적은 지역, dtype: float64

In [25]:
pca_result.to_csv(f'./data_to_model/data_after_pca_with_{scale_kind}_scaled_{len(pca_result.columns)}_features.csv')